## Monte Carlo Simulation


In [1]:
import random
import math

### Pi Calculation

In [ ]:
def pi_estimator(num_points):
    points_in_circle = 0

    for _ in range(num_points):
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)

        distance = math.sqrt(x**2 + y**2)

        if distance <= 1:
            points_in_circle += 1

    ratio= points_in_circle / num_points
    pi = 4 * (ratio)
    return pi

sample_sizes  = [10**3, 10**4, 10**5, 10**6,10**7]

for num_points in sample_sizes :
    pi_approx = pi_estimator(num_points)
    print(f"Number of points: {num_points} || Pi approximation: {pi_approx}")


The code utilizes a Monte Carlo simulation technique to estimate the value of π. This involves generating random points within a quarter circle inscribed in a square. The pi_estimator function is responsible for calculating the ratio of points within the circle, providing approximate values for π based on different quantities of generated points (ranging from ${10^3}$ to ${10^7}$). Notably, as the number of points increases, the precision of the π approximation improves. This highlight the method's effectiveness in converging towards the true value of Pi as the sample size grows.

In mathematical terms, the area of a square (As) is 4, and the area of a circle (Ac) can be represented by π. Therefore, the value of π can be expressed as 4 * (π/4) (Ac/As), where Ac is the area of the circle, and As is the area of the square.

### Mensch Game